In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

In [3]:
response = llm.invoke("Hello! Can you tell me a short joke in indian context about startup founders?")
print(response.content)

Here's one:

Why did the Indian startup founder bring a ladder to the funding meeting?

Because he wanted to take his valuation to a higher level, and also to show the investors that his growth trajectory is "upwardly mobile" - just like his aunt's son who got a job in the US!


### Search node

In [7]:
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=3)
web_search_results = web_search.invoke("who is the Chief Minister of bangalore?")

print(web_search_results["results"][0]["content"])

Bengaluru Traffic, Wipro, Azim Premji: Karnataka Chief Minister Siddaramaiah Turns To Wipro As Bengaluru Traffic Worsens.


In [8]:
from langgraph.prebuilt import create_react_agent

groq_llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
)

# 2. Create BetterMeals research agent using Groq model
research_agent = create_react_agent(
    model=groq_llm,
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent",
)


In [9]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [11]:
for chunk in research_agent.stream(
    {"messages": [{"role": "user", "content": "who is the Chief Minister of bangalore?"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: research_agent
Tool Calls:
  tavily_search (fc_1c29d050-adf4-4756-b5da-aef094ef3f8c)
 Call ID: fc_1c29d050-adf4-4756-b5da-aef094ef3f8c
  Args:
    query: Chief Minister of Karnataka 2024
    search_depth: advanced


Update from node tools:


================================= Tool Message =================================
Name: tavily_search

{"query": "Chief Minister of Karnataka 2024", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://vajiramandravi.com/current-affairs/chief-ministers-of-karnataka/", "title": "Chief Ministers of Karnataka, List from 1947 to 2025, Tenure", "content": "→ UPSC Toppers List 2024 is released now. Shakti Dubey is UPSC AIR 1 2024 Topper.\n\n→ Also check Best IAS Coaching in Delhi\n\n## Chief Ministers of Karnataka FAQs\n\nQ1. Who is the current Chief Minister of Karnataka?+\n\nAns. The current Chief Minister o

In [12]:
from typing import TypedDict, Optional

class State(TypedDict):
    raw_message: str
    query_type: Optional[str]
    response: Optional[str]


In [28]:
def handle_meal_doubt(state: State):
    """Answer nutrition/meal-related user doubts."""
    return {"response": f"[MealDoubtAgent] Answering: {state['raw_message']}"}

def handle_user_update(state: State):
    """Update user preferences or nutrition plan."""
    return {"response": f"[UserUpdateAgent] Updating plan with: {state['raw_message']}"}

def handle_cook_update(state: State):
    """Handle cook updates like missing ingredients."""
    return {"response": f"[CookUpdateAgent] Cook update received: {state['raw_message']}"}

def handle_grocery_order(state: State):
    """Trigger or update grocery orders."""
    return {"response": f"[GroceryOrderAgent] Ordering groceries: {state['raw_message']}"}


In [29]:
supervisor_agent = create_react_agent(
    model=groq_llm,
    tools=[
        handle_meal_doubt,
        handle_user_update,
        handle_cook_update,
        handle_grocery_order,
    ],
    prompt=(
        "You are the Supervisor Agent for BetterMeals.\n"
        "- Route meal/nutrition doubts to MealDoubtAgent.\n"
        "- Route user profile or preference changes to UserUpdateAgent.\n"
        "- Route cook updates (missing ingredients) to CookUpdateAgent.\n"
        "- Route grocery ordering requests to GroceryOrderAgent.\n"
        "Always assign tasks to the correct agent. Do not solve them yourself."
    ),
    name="supervisor",
)

In [31]:
for chunk in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": "order brocoli for dinner?"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  handle_grocery_order (fc_4627a708-183e-4e84-b1f5-9eb0ffd5803a)
 Call ID: fc_4627a708-183e-4e84-b1f5-9eb0ffd5803a
  Args:
    state: {'query_type': 'grocery_order', 'raw_message': 'order brocoli for dinner?', 'response': ''}


Update from node tools:


================================= Tool Message =================================
Name: handle_grocery_order

{"response": "[GroceryOrderAgent] Ordering groceries: order brocoli for dinner?"}


Update from node agent:


================================== Ai Message ==================================
Name: supervisor

[GroceryOrderAgent] Ordering groceries: order brocoli for dinner?


